In [35]:
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
META_DIR = Path('data/fma_metadata')
AUDIO_DIR = Path('data/fma_small')

# Helper funcitons

In [24]:
# From https://github.com/mdeff/fma/blob/master/utils.py

def get_audio_path(audio_dir, track_id):
    """
    Return the path to the mp3 given the directory where the audio is stored
    and the track ID.
    Examples
    --------
    >>> import utils
    >>> AUDIO_DIR = os.environ.get('AUDIO_DIR')
    >>> utils.get_audio_path(AUDIO_DIR, 2)
    '../data/fma_small/000/000002.mp3'
    """
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')


def get_tids_from_directory(audio_dir):
    """Get track IDs from the mp3s in a directory.
    Parameters
    ----------
    audio_dir : str
        Path to the directory where the audio files are stored.
    Returns
    -------
        A list of track IDs.
    """
    tids = []
    for _, dirnames, files in os.walk(audio_dir):
        if dirnames == []:
            tids.extend(int(file[:-4]) for file in files)
    return tids


# Based on https://github.com/priya-dwivedi/Music_Genre_Classification/

def create_spectrogram(track_id):
    filename = get_audio_path(AUDIO_DIR, track_id)
    y, sr = librosa.load(filename)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=1024)
    spect = librosa.power_to_db(spect, ref=np.max)
    return spect.T

def plot_spect(track_id):
    spect = create_spectrogram(track_id)
    print(spect.shape)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spect.T, y_axis='mel', fmax=8000, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.show()
    
def create_array(df, genre_dict):
    genres = []
    X_spect = np.empty((0, 640, 128))
    count = 0
    #Code skips records in case of errors
    for index, row in df.iterrows():
        try:
            count += 1
            track_id = int(row['track_id'])
            genre = str(row[('track', 'genre_top')])
            spect = create_spectrogram(track_id)

            # Normalize for small shape differences
            spect = spect[:640, :]
            X_spect = np.append(X_spect, [spect], axis=0)
            genres.append(genre_dict[genre])
            if count % 100 == 0:
                print("Currently processing: ", track_id)
        except:
#             print("Couldn't process: ", track_id)
            continue
    y_arr = np.array(genres)
    return X_spect, y_arr

# Get metadata

In [9]:
tracks = pd.read_csv(META_DIR/'tracks.csv', index_col=0, header=[0, 1]) 

# Keep necessary columns
keep_cols = [('set', 'split'), ('set', 'subset'),('track', 'genre_top')]
df_all = tracks[keep_cols]

# Use small dataset
df_all = df_all[df_all[('set', 'subset')] == 'small']

# Move index to track_id column
df_all['track_id'] = df_all.index

## Trim down to 2 genres

In [11]:
rock = df_all[('track', 'genre_top')] == 'Rock'
inst = df_all[('track', 'genre_top')] == 'Instrumental'
mask = rock | inst

two_genre = df_all[mask]

In [14]:
genre_dict = {'Rock': 1, 'Instrumental': 2}

## Create train, validation and test subsets

In [25]:
df_train = two_genre[two_genre[('set', 'split')]=='training']
df_valid = two_genre[two_genre[('set', 'split')]=='validation']
df_test = two_genre[two_genre[('set', 'split')]=='test']

print(df_train.shape, df_valid.shape, df_test.shape)

(1600, 4) (200, 4) (200, 4)


In [33]:
df_train.head()

set            track track_id
             split subset genre_top         
track_id                                    
255       training  small      Rock      255
256       training  small      Rock      256
368       training  small      Rock      368
574       training  small      Rock      574
777       training  small      Rock      777

# Create Spectrogram Arrays

In [37]:
X_train, y_train = create_array(df_train, genre_dict)

In [38]:
X_train.shape

(7, 640, 128)

In [39]:
y_train.shape

(7,)